# BM25 Retriever
In this guide, we define a bm25 retriever that search documents using bm25 method.

This notebook is very similar to the RouterQueryEngine notebook.

## Setup

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [5]:
import os
import logging
import sys

# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.
import nest_asyncio

from llama_index.core import (
    SimpleDirectoryReader,
    StorageContext,
    VectorStoreIndex,
    Settings,
    Document,
    QueryBundle,
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.schema import NodeWithScore
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.core.tools import RetrieverTool

from llama_index.core.retrievers import (
    BaseRetriever,
    RouterRetriever,
    VectorIndexRetriever,
)
from llama_index.retrievers.bm25 import BM25Retriever

from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [ ]:
nest_asyncio.apply()

os.environ["GROQ_API_KEY"] = "api key"

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().handlers = []
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Download Data

In [6]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

--2024-06-24 05:45:19--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘data/paul_graham/paul_graham_essay.txt’

data/paul_graham/pa 100%[===================>]  73.28K  --.-KB/s    in 0.08s   

2024-06-24 05:45:20 (926 KB/s) - ‘data/paul_graham/paul_graham_essay.txt’ saved [75042/75042]



## Load Data

We first show how to convert a Document into a set of Nodes, and insert into a DocumentStore.

In [7]:
# load documents
documents = SimpleDirectoryReader("./data/paul_graham").load_data()

In [8]:
# initialize LLM + node parser
llm = Groq(model="llama3-70b-8192", api_key=os.environ["GROQ_API_KEY"])
splitter = SentenceSplitter(chunk_size=1024)

nodes = splitter.get_nodes_from_documents(documents)

In [9]:
# initialize storage context (by default it's in-memory)
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

In [10]:
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/distiluse-base-multilingual-cased-v1")

Settings.embed_model = embed_model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
index = VectorStoreIndex(
    nodes=nodes,
    storage_context=storage_context,
)

## BM25 Retriever

We will search document with bm25 retriever.

In [12]:
# We can pass in the index, doctore, or list of nodes to create the retriever
retriever = BM25Retriever.from_defaults(nodes=nodes, similarity_top_k=2)

In [13]:
# will retrieve context from specific companies
nodes = retriever.retrieve("What happened at Viaweb and Interleaf?")
for node in nodes:
    display_source_node(node)

**Node ID:** ee67c258-72d3-4894-9402-312f5293e86a<br>**Similarity:** 0.9211411108145796<br>**Text:** Now that I could write essays again, I wrote a bunch about topics I'd had stacked up. I kept writ...<br>

**Node ID:** 60b9a2d7-5076-4454-ba4b-973490d0c208<br>**Similarity:** 1.5473312198012366<br>**Text:** I couldn't have put this into words when I was 18. All I knew at the time was that I kept taking ...<br>

In [14]:
nodes = retriever.retrieve("What did Paul Graham do after RISD?")
for node in nodes:
    display_source_node(node)

**Node ID:** b8aeded2-6dff-4e25-84ec-cad2a4b1b208<br>**Similarity:** 0.0<br>**Text:** It was this that attracted me in college, though I didn't understand why at the time.

McCarthy's...<br>

**Node ID:** 4e6e8040-c43c-41e3-a23b-2a5daac621ed<br>**Similarity:** 0.0<br>**Text:** Painting students were supposed to express themselves, which to the more worldly ones meant to tr...<br>

## Router Retriever with bm25 method

Now we will combine bm25 retriever with vector index retriever.

In [15]:
vector_retriever = VectorIndexRetriever(index)
bm25_retriever = BM25Retriever.from_defaults(nodes=nodes, similarity_top_k=2)

retriever_tools = [
    RetrieverTool.from_defaults(
        retriever=vector_retriever,
        description="Useful in most cases",
    ),
    RetrieverTool.from_defaults(
        retriever=bm25_retriever,
        description="Useful if searching about specific information",
    ),
]

In [16]:
retriever = RouterRetriever.from_defaults(
    retriever_tools=retriever_tools,
    llm=llm,
    select_multi=True,
)

In [18]:
# will retrieve all context from the author's life
a = retriever.retrieve(
    "Can you give me all the context regarding the author's life?"
)
for b in a:
    display_source_node(b)

ValidationError: 1 validation error for NodeWithScore
node
  Can't instantiate abstract class BaseNode with abstract methods get_content, get_metadata_str, get_type, hash, set_content (type=type_error)

## Advanced - Hybrid Retriever + Re-Ranking

Here we extend the base retriever class and create a custom retriever that always uses the vector retriever and BM25 retreiver.

Then, nodes can be re-ranked and filtered. This lets us keep intermediate top-k values large and letting the re-ranking filter out un-needed nodes.

To best demonstrate this, we will use a larger set of source documents -- Chapter 3 from the 2022 IPCC Climate Report.

### Setup data

In [19]:
!curl https://www.ipcc.ch/report/ar6/wg2/downloads/report/IPCC_AR6_WGII_Chapter03.pdf --output IPCC_AR6_WGII_Chapter03.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 20.7M  100 20.7M    0     0  16.7M      0  0:00:01  0:00:01 --:--:-- 16.7M


In [21]:
# load documents
documents = SimpleDirectoryReader(
    input_files=["IPCC_AR6_WGII_Chapter03.pdf"]
).load_data()

In [22]:
# initialize llm + node parser
# -- here, we set a smaller chunk size, to allow for more effective re-ranking
llm = Groq(model="llama3-70b-8192", api_key=os.environ["GROQ_API_KEY"])
splitter = SentenceSplitter(chunk_size=256)
# limit to a smaller section
nodes = splitter.get_nodes_from_documents(
    [Document(text=documents[0].get_content()[:1000000])]
)

In [23]:
# initialize storage context (by default it's in-memory)
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

In [24]:
index = VectorStoreIndex(nodes, storage_context=storage_context)

In [25]:
# retireve the top 10 most similar nodes using embeddings
vector_retriever = index.as_retriever(similarity_top_k=10)

# retireve the top 10 most similar nodes using bm25
bm25_retriever = BM25Retriever.from_defaults(nodes=nodes, similarity_top_k=10)

### Custom Retriever Implementation

In [26]:
class HybridRetriever(BaseRetriever):
    def __init__(self, vector_retriever, bm25_retriever):
        self.vector_retriever = vector_retriever
        self.bm25_retriever = bm25_retriever
        super().__init__()

    def _retrieve(self, query, **kwargs):
        bm25_nodes = self.bm25_retriever.retrieve(query, **kwargs)
        vector_nodes = self.vector_retriever.retrieve(query, **kwargs)

        # combine the two lists of nodes
        all_nodes = []
        node_ids = set()
        for n in bm25_nodes + vector_nodes:
            if n.node.node_id not in node_ids:
                all_nodes.append(n)
                node_ids.add(n.node.node_id)
        return all_nodes

In [27]:
index.as_retriever(similarity_top_k=5)

hybrid_retriever = HybridRetriever(vector_retriever, bm25_retriever)

### Re-Ranker Setup

In [29]:
reranker = SentenceTransformerRerank(top_n=4, model="unicamp-dl/monoptt5-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at unicamp-dl/monoptt5-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if y

### Retrieve

In [30]:
retrieved_nodes = hybrid_retriever.retrieve(
    "What is the impact of climate change on the ocean?"
)
reranked_nodes = reranker.postprocess_nodes(
    retrieved_nodes,
    query_bundle=QueryBundle(
        "What is the impact of climate change on the ocean?"
    ),
)

print("Initial retrieval: ", len(retrieved_nodes), " nodes")
print("Re-ranked retrieval: ", len(reranked_nodes), " nodes")

Initial retrieval:  13  nodes
Re-ranked retrieval:  4  nodes


In [31]:
for node in reranked_nodes:
    display_source_node(node)

**Node ID:** 50cf7b5d-6994-41a0-9bb3-eb2cebc3dc2b<br>**Similarity:** 0.580895721912384<br>**Text:** Stephanie Dutkiewicz (USA), Thomas Frölicher 
(Switzerland), Juan Diego Gaitán-Espitia (Hong Kong...<br>

**Node ID:** 03ddd1cb-1a3d-41d2-ad7d-5894910c7e89<br>**Similarity:** 0.5701732635498047<br>**Text:** Ghebrehiwet, S.-I.  Ito, W.  Kiessling, P .  Martinetto, E.  Ojea, 
M.-F . Racault, B.  Rost, and...<br>

**Node ID:** 1dcb88f6-6d4c-40ff-bf4d-45a37d4985a7<br>**Similarity:** 0.568600594997406<br>**Text:** Pörtner, D.C.  Roberts, M.  Tignor, E.S.  Poloczanska, K.  Mintenbeck, 
A. Alegría, M.  Craig, S....<br>

**Node ID:** 067f6634-43c2-46d6-a732-d558557fda84<br>**Similarity:** 0.566605806350708<br>**Text:** In: Climate 
Change 2022: Impacts, Adaptation and Vulnerability. Contribution of Working Group II...<br>

### Full Query Engine

In [32]:
query_engine = RetrieverQueryEngine.from_args(
    retriever=hybrid_retriever,
    node_postprocessors=[reranker],
    llm=llm,
)

response = query_engine.query(
    "What is the impact of climate change on the ocean?"
)

In [33]:
display_response(response)

**`Final Response:`** The impact of climate change on the ocean is addressed in the chapter "Oceans and Coastal Ecosystems and Their Services" in the report "Climate Change 2022: Impacts, Adaptation and Vulnerability".